setup

In [1]:
import json
import pandas as pd
import get_audio_features as gaf
import get_token as gt 
from requests import post, get

token = gt.get_token()

import json file as a dataframe

In [2]:
tracks_df = pd.read_json('tracks_w_ids.json')
print(len(tracks_df))
tracks_df.head()

6416


,endTime,artistName,trackName,msPlayed,trackId
0,2023-03-18 21:28,Eudes Alvarez,San Rafael de Mi Pueblo,16688,7A80n2HvAs1rZNNs64NN8F
1,2023-03-18 23:52,Sam Cooke,Another Saturday Night,153052,5z8DiKSG4EwxmO6gUi12rZ
2,2023-03-19 19:33,Zoom Soon Bao,Reminiscencia - Remix,2365,0
3,2023-03-19 19:34,Soledad Bravo,Violín de becho,13552,7hkyFS309jQBrwrbxGsJuH
4,2023-03-19 19:37,Soledad Bravo,Violín de becho,196288,7hkyFS309jQBrwrbxGsJuH


change data type of endtime column to datetime64 so I can then format the date as needed (TRANSFORMATION)

In [3]:
tracks_df['endTime'] = pd.to_datetime(tracks_df['endTime'])
tracks_df['streamingDate'] = tracks_df['endTime'].dt.strftime('%Y-%m-%d')
tracks_df.head()

,endTime,artistName,trackName,msPlayed,trackId,streamingDate
0,2023-03-18 21:28:00,Eudes Alvarez,San Rafael de Mi Pueblo,16688,7A80n2HvAs1rZNNs64NN8F,2023-03-18
1,2023-03-18 23:52:00,Sam Cooke,Another Saturday Night,153052,5z8DiKSG4EwxmO6gUi12rZ,2023-03-18
2,2023-03-19 19:33:00,Zoom Soon Bao,Reminiscencia - Remix,2365,0,2023-03-19
3,2023-03-19 19:34:00,Soledad Bravo,Violín de becho,13552,7hkyFS309jQBrwrbxGsJuH,2023-03-19
4,2023-03-19 19:37:00,Soledad Bravo,Violín de becho,196288,7hkyFS309jQBrwrbxGsJuH,2023-03-19


remove rows with zero values in the trackId column

In [4]:
tracks_df2 = tracks_df[tracks_df['trackId'] != 0]
print(f"number of records: {len(tracks_df2)}")
tracks_df2.head()

number of records: 6157


,endTime,artistName,trackName,msPlayed,trackId,streamingDate
0,2023-03-18 21:28:00,Eudes Alvarez,San Rafael de Mi Pueblo,16688,7A80n2HvAs1rZNNs64NN8F,2023-03-18
1,2023-03-18 23:52:00,Sam Cooke,Another Saturday Night,153052,5z8DiKSG4EwxmO6gUi12rZ,2023-03-18
3,2023-03-19 19:34:00,Soledad Bravo,Violín de becho,13552,7hkyFS309jQBrwrbxGsJuH,2023-03-19
4,2023-03-19 19:37:00,Soledad Bravo,Violín de becho,196288,7hkyFS309jQBrwrbxGsJuH,2023-03-19
5,2023-03-19 19:39:00,Soledad Bravo,Canción para mi américa,113226,54a3QojIm00P4MXL02BZvC,2023-03-19


checking to see the correct number of rows remain after removing zero values

In [5]:
original_count = 6416
updated_count = 6157
null_trackids = 259
original_count - updated_count == null_trackids

True

### saving this transformed dataframe to json file

In [6]:
tracks_df2.to_json('tracks_wids_nozeroes.json')

create a dataframe that contains only the track ids, and then create a list with the numbers you'll use to slice the dataframe to create the batches.

In [7]:
# ids_only_df = tracks_df2['trackId']
ids_only_df = pd.DataFrame(tracks_df2['trackId'])
# print(len(ids_only_df))
fixed_batch = []
# for i in range (0, 65):
for i in range (0, 62):
    fixed_batch.append(i*100)
fixed_batch.append(6157)
# print(fixed_batch)
# len(fixed_batch)


This will ultimately create an array containing strings made up of 100 track ids concatenated with a comma as a delimiter.  Each element in the array is one of the batches.  

In [8]:

batch_array = []
# batch_row = []
for j in range (0, 62):
    batch_row = []
    # batch_name = "batch"+ str(j)
    start = fixed_batch[j]
    end = fixed_batch[j+1]
    # print(f"j = {j} and range for {batch_name} is from {start} to {end}")
    batch = ids_only_df.iloc[start:end] # retrieve batch of data
    batch_str = "" # create empty string to hold comma separated ids
    for index, row in batch.iterrows(): # iterate through each row in batch and append trackId to a list (batch_row)
        batch_row.append(row['trackId'])
    delimiter = ","
    batch_str = delimiter.join(batch_row) # create a string that will contain comma separated track ids
    batch_array.append(batch_str) # append string to batch_array
        
len(batch_array)
    


62

checking the array to make sure everything was concatenated and stored correctly

In [9]:
# print(batch_array[9])
# test = batch_array[9].split(",")
# print(test)
# len(test)


## Call API to get audio features for each track

create a function that will append json data to an existing json file

In [10]:
def append_to_json(new_data, filename):
    with open(filename, 'a') as file:
        json.dump(new_data, file, indent=4)

test the output of extract_many_audio_features function

In [11]:
# features = gaf.extract_many_audio_features(token, batch_array[9])
# print(features)
# print(len(features))
# print(len(batch_array[9]))d
# print(len(batch_array))

iterate through array of concatenated ids, send them to function to get their audio features, and append the audio feature json data to a json file

In [12]:
batch_count = 0
for ids_string in batch_array:
    features = gaf.extract_many_audio_features(token, ids_string)
    # features2 = features["audio_features"]
    print(f"received data for row {batch_count}.")
    append_to_json(features, 'raw_audio_features.json')
    # append_to_json(",", 'raw_audio_features.json')
    batch_count += 1

received data for row 0.
received data for row 1.
received data for row 2.
received data for row 3.
received data for row 4.
received data for row 5.
received data for row 6.
received data for row 7.
received data for row 8.
received data for row 9.
received data for row 10.
received data for row 11.
received data for row 12.
received data for row 13.
received data for row 14.
received data for row 15.
received data for row 16.
received data for row 17.
received data for row 18.
received data for row 19.
received data for row 20.
received data for row 21.
received data for row 22.
received data for row 23.
received data for row 24.
received data for row 25.
received data for row 26.
received data for row 27.
received data for row 28.
received data for row 29.
received data for row 30.
received data for row 31.
received data for row 32.
received data for row 33.
received data for row 34.
received data for row 35.
received data for row 36.
received data for row 37.
received data for row 

For this next step, I first had to manually go into the raw_audio_features json file and do a search and replace for ][ and , respectively to get json to be formatted correctly.  I'm not sure how to do this programatically.  Using the json_data.replace() function introduces a whole bunch of white space characters into the file. 

Load json file into dataframe

In [21]:
pleasework = pd.read_json('raw_audio_features.json')
pleasework.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6157.000000,6.157000e+03,6157.000000
mean,0.517081,0.432510,5.181582,-12.224055,0.680201,0.054788,0.560518,0.243062,0.172352,0.458772,115.353670,2.122584e+05,3.798766
std,0.185971,0.256007,3.505658,6.520114,0.466436,0.064905,0.337198,0.375634,0.156729,0.293234,31.716705,1.932044e+05,0.594220
min,0.000000,0.000000,0.000000,-44.211000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.151000e+03,0.000000
25%,0.393000,0.221000,2.000000,-14.753000,0.000000,0.032500,0.229000,0.000000,0.094200,0.200000,92.945000,1.540000e+05,4.000000
50%,0.534000,0.416000,5.000000,-10.694000,1.000000,0.039200,0.638000,0.000459,0.113000,0.413000,114.229000,1.964280e+05,4.000000
75%,0.652000,0.633000,8.000000,-7.844000,1.000000,0.051900,0.871000,0.613000,0.181000,0.717000,134.355000,2.440670e+05,4.000000
max,0.949000,0.995000,11.000000,0.291000,1.000000,0.962000,0.996000,1.000000,1.000000,0.997000,227.959000,4.320028e+06,5.000000


In [22]:
pleasework.head(20)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.3750,0.728,9,-10.130,0,0.0433,0.759000,0.860000,0.1410,0.9010,118.414,audio_features,7A80n2HvAs1rZNNs64NN8F,spotify:track:7A80n2HvAs1rZNNs64NN8F,https://api.spotify.com/v1/tracks/7A80n2HvAs1r...,https://api.spotify.com/v1/audio-analysis/7A80...,177800,4
1,0.7760,0.659,9,-7.767,1,0.0510,0.434000,0.000000,0.0868,0.9690,123.094,audio_features,5z8DiKSG4EwxmO6gUi12rZ,spotify:track:5z8DiKSG4EwxmO6gUi12rZ,https://api.spotify.com/v1/tracks/5z8DiKSG4Ewx...,https://api.spotify.com/v1/audio-analysis/5z8D...,160000,4
2,0.4260,0.300,11,-9.717,0,0.0365,0.573000,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,spotify:track:7hkyFS309jQBrwrbxGsJuH,https://api.spotify.com/v1/tracks/7hkyFS309jQB...,https://api.spotify.com/v1/audio-analysis/7hky...,209840,4
3,0.4260,0.300,11,-9.717,0,0.0365,0.573000,0.000011,0.1270,0.5250,116.445,audio_features,7hkyFS309jQBrwrbxGsJuH,spotify:track:7hkyFS309jQBrwrbxGsJuH,https://api.spotify.com/v1/tracks/7hkyFS309jQB...,https://api.spotify.com/v1/audio-analysis/7hky...,209840,4
4,0.5550,0.293,3,-8.525,0,0.0470,0.877000,0.000001,0.1180,0.1130,140.935,audio_features,54a3QojIm00P4MXL02BZvC,spotify:track:54a3QojIm00P4MXL02BZvC,https://api.spotify.com/v1/tracks/54a3QojIm00P...,https://api.spotify.com/v1/audio-analysis/54a3...,113227,3
5,0.6140,0.277,9,-12.052,1,0.1200,0.852000,0.000000,0.1060,0.5810,122.659,audio_features,1ebslsQcXFPNRBLsa76qEK,spotify:track:1ebslsQcXFPNRBLsa76qEK,https://api.spotify.com/v1/tracks/1ebslsQcXFPN...,https://api.spotify.com/v1/audio-analysis/1ebs...,180082,4
6,0.1370,0.119,1,-23.390,1,0.0575,0.978000,0.940000,0.1010,0.0374,70.023,audio_features,41F8XFf9aYq1vCMqyuLNFl,spotify:track:41F8XFf9aYq1vCMqyuLNFl,https://api.spotify.com/v1/tracks/41F8XFf9aYq1...,https://api.spotify.com/v1/audio-analysis/41F8...,336001,4
7,0.8180,0.151,9,-21.946,0,0.0718,0.917000,0.917000,0.1110,0.4140,142.153,audio_features,6OzRUo7b23kXFXHWLpigCj,spotify:track:6OzRUo7b23kXFXHWLpigCj,https://api.spotify.com/v1/tracks/6OzRUo7b23kX...,https://api.spotify.com/v1/audio-analysis/6OzR...,121690,4
8,0.5660,0.237,6,-17.012,0,0.0282,0.398000,0.897000,0.1160,0.0761,80.020,audio_features,61liR3eWrA8BTeUnltA32e,spotify:track:61liR3eWrA8BTeUnltA32e,https://api.spotify.com/v1/tracks/61liR3eWrA8B...,https://api.spotify.com/v1/audio-analysis/61li...,147000,4
9,0.6940,0.337,5,-12.300,0,0.0624,0.609000,0.597000,0.1760,0.5690,126.235,audio_features,601ppwJ942tjJ9aDPGYwOT,spotify:track:601ppwJ942tjJ9aDPGYwOT,https://api.spotify.com/v1/tracks/601ppwJ942tj...,https://api.spotify.com/v1/audio-analysis/601p...,190476,4
